In [ ]:
import torch, torchvision
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from build_model import ResNet50, ResNet101, Swin_T
from engine import MyTrainer
from read_data import TongueDataset

In [ ]:
epochs = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = pd.read_excel("Shexiang-2024-06.23.xlsx")


train_data, test_data = train_test_split(data, test_size=0.1,random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.125,random_state=42)
train_dataset = TongueDataset(train_data, transform=TongueDataset.get_tranforms(train=True))
val_dataset = TongueDataset(val_data, transform=TongueDataset.get_tranforms(train=False))
test_dataset = TongueDataset(test_data, transform=TongueDataset.get_tranforms(train=False))

train_loader = DataLoader(dataset=train_dataset,batch_size=64,shuffle=False)
val_loader = DataLoader(dataset=val_dataset,batch_size=64,shuffle=False)
test_loader = DataLoader(dataset=val_dataset,batch_size=64,shuffle=False)



net = ResNet50(out_features=1, pretrained=True)
net = net.to(device)
net.load_state_dict(torch.load("checkpoints/best.pth"))


criterion = nn.BCELoss()


In [ ]:
val_outputs = []
val_labels = []
net.eval()
with torch.no_grad():

    for images, targets in test_loader:
        images, targets = images.to(device),targets.to(device)
        outputs = net(images)

        val_outputs.extend(outputs.squeeze().detach().cpu().numpy().tolist())
        val_labels.extend(targets.squeeze().detach().cpu().numpy().tolist())



In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
# 假设的y_pred和y_label数组

# 计算校准曲线
prob_true, prob_pred = calibration_curve(val_labels, val_outputs, n_bins=10)
# 绘制校准曲线
plt.figure(figsize=(10, 5))
plt.plot(prob_pred, prob_true, marker='o', linestyle='-', color='b')
plt.plot([0, 1], [0, 1], linestyle='--', color='r')  # 对角线
plt.xlabel('预测概率')
plt.ylabel('实际阳性比例')
plt.title('校准曲线')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(val_labels, val_outputs)  # 假设你的目标变量是二分类问题，并且正类标签为1
roc_auc = auc(fpr, tpr)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
outputs_np = np.array(val_outputs)
outputs_np[outputs_np>=0.5] = 1
outputs_np[outputs_np<0.5] = 0
labels_np = np.array(val_labels)
print(classification_report(labels_np, outputs_np))